# Missing Data And Timing Jitter

During Launch-12 several flight comptuer events were recorded that indicated a loss of data.

## Sequence Numbers And Ethernet

All data sent from the instruments in the rocket are sent over physical ethernet cables to a network switch on the top of the stack, then the main computer recieves them. In the flight executive incoming packets are recieved and the high resolution timestamp of the exact recieve time recoreded.

[Ethmux code from the FC in the av3-fc repo](https://github.com/psas/av3-fc/blob/master/src/ethmux.c#L81)

All the sequential data, like from and IMU or other device polled on an interval, is sent over the network with a sequence number so we can detect missing data on receipt. Skipped sequence numbers are recoreded by the `SEQE` ("Sequence Error") definition and recorded in the log.

We did not expect to miss any data. The network stack should have been streaming well below it's theoretical limit. However when checking the logs in for SEQE Data, we see this:

In [ ]:
with open("../fc-data/SEQE.csv") as f_in:
    print f_in.read()

At several times we get a sequence error from multiple ports. In each case there is a chunk of missing data.

The incoming port numbers can be decoded as follows:

 - 36104: RNH_UMBDET (Umbilical disconnect signal)
 - 35050: JGPS_PORT (Raw GPS data)
 - 36102: RNH_PORT (Battery and power data)
 - 35020: ADIS_PORT (IMU data)
 - 35011: BMP_PORT (Pressure data)
 - 35051: GPS_COTS (Standard, comercial GPS data)
 
In the flight there were two significant upset events where we lost at least one or more packets of data from nearly every sensor on the rocket(!)

## Lost milliseconds In IMU Data

If we chart the difference in recieve time between each and every sample of IMU data we should see these two gaps show up (log scale to see the baseline and the huge jumps):

In [ ]:
from numpy import loadtxt, array, subtract, divide, multiply, median, std, var
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
#set_matplotlib_formats('png', 'pdf')

# Liftoff Time
t_0 = 117853569585227

# Import IMU Data
columns = loadtxt("../fc-data/ADIS.csv", delimiter=',', unpack=True, comments="#")
seqn = columns[0]
timestamp = columns[1]
gyro_x = columns[3]
gyro_y = columns[4]
gyro_z = columns[5]
acc_x = columns[6]
acc_y = columns[7]
acc_z = columns[8]

# Put time in seconds
timestamp = subtract(timestamp, t_0)
timestamp = divide(timestamp, 1e9)

In [ ]:
diff = []
for i in range(1,len(timestamp)):
    diff.append(timestamp[i] - timestamp[i - 1])
ax = plt.figure(figsize=(16,10))
plt.title(r"Looking For Gaps In The IMU Timestamps")
plt.ylabel(r"Time Since Last Sample [ms]")
plt.xlabel(r"MET [s]")
plt.plot(timestamp[1:], multiply(diff, 1000))
ax.axes[0].set_yscale("log", nonposy='clip')
ax.axes[0].set_xlim([-5,42])
#ax.axes[0].set_ylim([-10,10])
plt.show()

The expeced time between samples from the ADIS IMU is 1.22 ms. However there are two big gaps in the data:

In [ ]:
for i, d in enumerate(diff):
    if d > 0.01:
        print "At time %6.3f s: %6.1f ms" % (timestamp[i+1], d*1000.0)

## Average Sample Time

If we ignore the bumps we should learn something about the jitter in the recieved sample time. The assumption is that the actual sample time was pretty uniform, and most of the jitter is from uncertaninty in the network stack and non-realtime OS that the flight computer is running.

Here we chart (linear scale this time) just the non-gap diffs

In [ ]:
diff = []
difftimes = []
for i in range(1,len(timestamp)):
    d = timestamp[i] - timestamp[i - 1]
    if d < 0.01:
        difftimes.append(timestamp[i])
        diff.append(d)

d_median = median(diff)
ax = plt.figure(figsize=(16,10))
plt.title(r"IMU Sample Times")
plt.ylabel(r"Time Since Last Sample [ms]")
plt.xlabel(r"MET [s]")
plt.plot(difftimes, multiply(diff, 1000), 'r.', alpha=0.35)
ax.axes[0].set_xlim([-5,42])
ax.axes[0].set_ylim([0.8,1.8])
plt.show()

## Jitter

In [ ]:
ax = plt.figure(figsize=(16,10))
plt.title(r"IMU Sample Times")
plt.ylabel(r"Sample Timeing Frequency")
plt.xlabel(r"Time Since Last Sample [ms]")
n, bins, patches = plt.hist(multiply(diff, 1000), 1000, histtype='step', normed=1, alpha=0.8, linewidth=1, fill=True)
plt.plot((d_median*1000.0, d_median*1000.0), (0, 6), 'b-', label="Median Sample Time")
plt.plot((1.2207, 1.2207), (0, 6), 'k-', label="True Sample Time")
plt.legend(loc=1)
#ax.axes[0].set_xlim([-5,42])
#ax.axes[0].set_ylim([0.8,1.8])
plt.show()

It's not gaussian. In fact the bump at almost exactly 1 ms is hard to explain.

Clearly there is some lower bound for timing. One might suspect that almost every time a packet arrives slightly late, the next one is likely to be on time (regression to the mean) so that there are an abundance of "short" arrivals that appear to get here quicker than they should. Why that number should be so close to 1.0 ms is hard to say.

Here are some statistical numbers for the jitter:

In [ ]:
print "True Sample Time:          1.221 ms"
print "Median Actual Sample Time: %5.3f ms" % (d_median*1000.0)
print "Standard Deviation:        %5.3f ms" % (std(diff)*1e3)